# Send my predictions to the boys
This system functions as a seamless pipeline that performs a series of intriguing operations, the primary of which is sourcing the current week's Footy Tipper predictions. With this information in hand, the pipeline proceeds to the next stage where it employs the advanced linguistic abilities of ChatGPT. Mimicking the persona of Reg Regan, the AI drafts an engaging and informed email containing the predictions. Upon completion, the email is dispatched to the boys, ensuring they are consistently kept in the loop with the latest Footy Tipper forecasts. The entire operation is a seamless fusion of sports analytics and artificial intelligence, creating an engaging and personalized user experience.

## Set up environment
The setup environment section of this project involves importing crucial libraries such as os, sys, sqlite3, pathlib, and pandas for various functionalities, along with dotenv for loading environment variables. Custom functions from the 'use-predictions' directory are also imported. Paths are established to the SQLite database, the secrets environment file, and a service account token file. Lastly, the script loads environment variables from the secrets file, enhancing security for sensitive elements.

In [1]:
# for the predictions
import os
import sys
import sqlite3
import pathlib
import pandas as pd
from dotenv import load_dotenv

# my funcitons
sys.path.append("functions") 
import sending_functions as sf

# Get to the root directory
project_root = pathlib.Path().absolute().parent.parent

# Now construct the relative path to your SQLite database
db_path = project_root / "data" / "footy-tipper-db.sqlite"
secrets_path = project_root / "secrets.env"
json_path = project_root / "service-account-token.json"

# lead secrets
load_dotenv(dotenv_path=secrets_path)

True

## Get prediciton data
For acquiring prediction data, the script begins by establishing a connection with the SQLite database using the established db_path. Once the connection is created, an SQL query is read from an external file named 'prediction_table.sql'. This query is designed to retrieve the necessary prediction data. The script then executes this query against the SQLite database and fetches the results into a Pandas DataFrame, predictions, enabling further data manipulation and analysis. Following the successful extraction of prediction data, the script closes the connection to the SQLite database to maintain good programming practice. The predictions dataframe is then displayed to review the retrieved data.

In [2]:
# Connect to the SQLite database
con = sqlite3.connect(str(db_path))

# Read SQL query from external SQL file
with open('sql/prediction_table.sql', 'r') as file:
    query = file.read()

# Execute the query and fetch the results into a data frame
predictions = pd.read_sql_query(query, con)

# Disconnect from the SQLite database
con.close()

predictions

,game_id,home_team_result,team_home,position_home,team_head_to_head_odds_home,team_away,position_away,team_head_to_head_odds_away,home_team_win_prob,home_team_lose_prob,round_id,competition_year,round_name
0,20231113010,Win,Penrith Panthers,1,1.19,Melbourne Storm,3,4.75,0.724130,0.275870,30,2023,Finals Week 3
1,20231113020,Win,Brisbane Broncos,2,1.26,New Zealand Warriors,4,3.90,0.632367,0.367633,30,2023,Finals Week 3


## Get Tipper Picks
For the crucial step of deriving the 'tipper picks', the function get_tipper_picks() from the previously imported 'sending_functions' module is used. This function operates on the predictions DataFrame, sifting through the data to identify the best market value based on the provided predictions. These selections, aptly named 'tipper picks', represent the optimal choices for those among the boys who are keen to place a punt. After the function has identified the 'tipper picks', these selections are assigned to the tipper_picks DataFrame for easy reference and further usage. This DataFrame is then displayed for review and verification.

In [3]:
tipper_picks = sf.get_tipper_picks(predictions)
tipper_picks

,team,price,price_min


## Save predicitons
In the 'Save Predictions' section, the predictions data, stored in the predictions DataFrame, is saved to Google Drive for record-keeping and future reference. This is accomplished using the upload_df_to_drive() function from the 'sending_functions' module. This function takes four arguments: the DataFrame to be uploaded (predictions), the path to the service account token (json_path), the Google Drive folder ID (which is securely accessed using os.getenv('FOLDER_ID') to fetch the environment variable), and the desired name of the file to be saved in the drive ("predictions.csv"). This process ensures that the valuable predictions data is not only used in the immediate context but also archived securely in Google Drive for potential future usage or analysis.

In [4]:
sf.upload_df_to_drive(
    predictions, 
    json_path, 
    os.getenv('FOLDER_ID'), 
    "predictions.csv"
)

File ID: 1z2C5FkNTJxAfkO8DBj1MJTFK78EE613f


## Reg R-ai-gan

In the 'Reg Regan Email Generation' phase, the system employs the sophisticated language model, ChatGPT, to draft an email encapsulating the week's footy predictions and tipper picks. The generate_reg_regan_email() function, hailing from the 'sending_functions' module, facilitates this operation. It accepts four arguments: the predictions and tipper_picks DataFrames, which contain the necessary information for the email content; the OpenAI API key (os.getenv('OPENAI_KEY')), fetched securely from the environment variables, which grants the ability to access the language model; and the URL of the Google Drive folder (os.getenv('FOLDER_URL')), to which the predictions.csv file was uploaded. The function compiles this information into an engaging, intelligently crafted email, channeling the persona of Reg Regan. Once the email is generated, it's assigned to the reg_regan variable and printed out for review before being sent to the boys.

In [5]:
reg_reagan = sf.generate_reg_regan_email(
    predictions, 
    tipper_picks, 
    os.getenv('OPENAI_KEY'), 
    os.getenv('FOLDER_URL'),
    1
)

print(reg_reagan)

/opt/homebrew/lib/python3.11/site-packages/langchain/llms/openai.py:173: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/opt/homebrew/lib/python3.11/site-packages/langchain/llms/openai.py:753: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


Subject: Reg Reagan's Top Picks for NRL Finals Week 3 2023

G'day Champions,

Reg Reagan here with this week's top footy picks, brought to you fresh from the digital brain of our beloved Footy Tipper. 

In the first match, our beloved machine learning system has predicted a win for the Penrith Panthers, currently sitting pretty at top position on the NRL ladder. They're up against the Melbourne Storm, who are in third place. The bookies seem pretty confident in the Panthers too with a head-to-head price of 1.19. As for Storm, they'll need more than their usual bluster to weather this one; the bookies are giving them an ambitious price of 4.75.

Moving on to the next bout, it seems like the Brisbane Broncos are about to trample all over the New Zealand Warriors. With Broncos firmly placed in second position and the Warriors in fourth, it seems like the bookies are favoring our home boys too, with a luscious head to head price of 1.26 whilst the Warriors are trailing at 3.9. 

In terms o

## Send emails
The final phase, 'Send Emails', sees the footy predictions being disseminated to the intended recipients. The send_emails() function from the 'sending_functions' module is invoked to accomplish this. It accepts several parameters: "footy-tipper-email-list", which is the list of recipients who will receive the email; a formatted string that serves as the email subject, dynamically including the round name from the predictions DataFrame; the reg_regan variable, which contains the email content; and the sender's email and password, fetched securely from the environment variables via os.getenv('MY_EMAIL') and os.getenv('EMAIL_PASSWORD') respectively. Finally, it takes the json_path, which might be required for email client authentication. This operation results in all the boys on the email list promptly receiving an email filled with the week's Footy Tipper predictions and tipper picks, all expressed in the unmistakable tone of Reg Regan.

In [6]:
sf.send_emails(
    "footy-tipper-email-list", 
    f"Footy Tipper Predictions for {predictions['round_name'].unique()[0]}", 
    reg_reagan, 
    os.getenv('MY_EMAIL'), 
    os.getenv('EMAIL_PASSWORD'), 
    json_path
)